https://www.nature.com/articles/s41591-022-02138-x

In [1]:
using RCall
figsize(;width,height) =
    RCall.ijulia_setdevice(MIME("image/svg+xml"), width=width, height=height)
R"
require(tidyverse)
"

┌ Warning: RCall.jl: Loading required package: tidyverse
│ ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
│ ✔ ggplot2 3.4.0     ✔ purrr   1.0.1
│ ✔ tibble  3.1.8     ✔ dplyr   1.1.0
│ ✔ tidyr   1.3.0     ✔ stringr 1.5.0
│ ✔ readr   2.1.3     ✔ forcats 1.0.0
│ ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
│ ✖ dplyr::filter() masks stats::filter()
│ ✖ dplyr::lag()    masks stats::lag()
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172


RObject{LglSxp}
[1] TRUE


In [2]:
R"
ct = read_csv('ct_data_cleaned.csv')
"

┌ Warning: RCall.jl: Rows: 76433 Columns: 35
│ ── Column specification ────────────────────────────────────────────────────────
│ Delimiter: ","
│ chr (14): Lineage, TestResult, AgeGroup, Diagnosis, VaccStatus, Symp_Init, S...
│ dbl (17): TestDate, CtT1, CtT2, DaysSinceDetection, DaysSinceNegative, LastN...
│ lgl  (4): TiterSensitivity, UseLessThan60, UseLessThan90, Use60to90
│ 
│ ℹ Use `spec()` to retrieve the full column specification for this data.
│ ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172


RObject{VecSxp}
# A tibble: 76,433 × 35
   Lineage TestDate TestRe…¹  CtT1  CtT2 AgeGr…² DaysS…³ DaysS…⁴ LastN…⁵ TimeR…⁶
   <chr>      <dbl> <chr>    <dbl> <dbl> <chr>     <dbl>   <dbl>   <dbl>   <dbl>
 1 B.1.427        0 Negative  40    40   (0,30]       -5       0       0     -10
 2 B.1.427        2 Negative  40    40   (0,30]       -3       0       2      -8
 3 B.1.427        3 Negative  40    40   (0,30]       -2       0       3      -7
 4 B.1.427        4 Negative  40    40   (0,30]       -1       0       4      -6
 5 B.1.427        5 Positive  34.8  38.7 (0,30]        0       1       4      -5
 6 B.1.427        6 Positive  33.9  36.4 (0,30]        1       2       4      -4
 7 B.1.427        7 Positive  25.6  26.5 (0,30]        2       3       4      -3
 8 B.1.427        8 Positive  24.5  25.0 (0,30]        3       4       4      -2
 9 B.1.427        9 Positive  26.4  27.2 (0,30]        4       5       4      -1
10 B.1.427       10 Positive  24.5  25.3 (0,30]        5       6     

In [3]:
figsize(width=6,height=5)
R"
dat = ct %>%
    filter(
        !(TestResult %in% c('No sample', 'Positive (no CT)')),
        LineageBroad %in% c('Delta', 'Omicron', 'Other')
    ) %>%
    {
        t = .
        t %>%
            filter(
                # any first positive must be preceded by negative test
                DaysSinceDetection==0 & !is.na(DaysSinceNegative),
                # at this day, Ct must be meaningful and above l.o.d.
                0 <= CtT1 & CtT1 < 40,
            ) %>%
            transmute(
                # we only use those samples
                PersonID, CumulativeInfectionNumber,
                DetectionSpeed = cut(DaysSinceNegative, breaks=c(0,3,Inf), right=FALSE)
            ) %>%
            left_join(t)
    }

ggplot(dat, aes(x=DaysSinceDetection)) +
    scale_y_reverse() +
    geom_jitter(
        aes(y=CtT1),
        data = dat, alpha=.1, size=.1
    ) +
    geom_line(
        aes(y=m),
        data = dat %>%
            group_by(DaysSinceDetection, LineageBroad, DetectionSpeed) %>%
            summarise(m=mean(CtT1, na.rm=TRUE))
    ) +
    geom_line(
        aes(y=m),
        data = dat %>%
            group_by(DaysSinceDetection, LineageBroad, DetectionSpeed) %>%
            summarise(m=median(CtT1, na.rm=TRUE)),
        linetype='dashed'
    ) +
    geom_text(
        aes(x=Inf,y=-Inf, label=label, hjust=1.1, vjust=1.1),
        data = dat %>%
            filter(DaysSinceDetection==0) %>%
            group_by(LineageBroad, DetectionSpeed) %>%
            tally() %>%
            mutate(label = paste('N =', n))
    ) +
    facet_grid(LineageBroad ~ DetectionSpeed) +
    lims(x=c(0,25), y=c(40, min(dat$CtT1, na.rm=TRUE)))
"

┌ Warning: RCall.jl: Joining with `by = join_by(PersonID, CumulativeInfectionNumber)`
│ Warning in left_join(., t) :
│   Each row in `x` is expected to match at most 1 row in `y`.
│ ℹ Row 1 of `x` matches multiple rows.
│ ℹ If multiple matches are expected, set `multiple = "all"` to silence this
│   warning.
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: `summarise()` has grouped output by 'DaysSinceDetection', 'LineageBroad'. You
│ can override using the `.groups` argument.
│ `summarise()` has grouped output by 'DaysSinceDetection', 'LineageBroad'. You
│ can override using the `.groups` argument.
│ Scale for y is already present.
│ Adding another scale for y, which will replace the existing scale.
└ @ RCall ~/.julia/packages/RCall/Wyd74/src/io.jl:172
┌ Warning: RCall.jl: Warning: Removed 25706 rows containing missing values (`geom_point()`).
│ Warning: Removed 38 rows containing missing values (`geom_line()`).
│ Warning: Removed 38 rows containing missing v

RObject{VecSxp}
